In [1]:
#import pandas as pd
!wget http://www-public.tem-tsp.eu/~zhang_da/pub/dataset_tsmc2014.zip
!unzip dataset_tsmc2014.zip

--2024-05-27 15:59:09--  http://www-public.tem-tsp.eu/~zhang_da/pub/dataset_tsmc2014.zip
Resolving www-public.tem-tsp.eu (www-public.tem-tsp.eu)... 157.159.10.107, 2001:660:3203:100:1:0:80:107
Connecting to www-public.tem-tsp.eu (www-public.tem-tsp.eu)|157.159.10.107|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25546284 (24M) [application/zip]
Saving to: ‘dataset_tsmc2014.zip’

dataset_tsmc2014.zi 100%[===================>]  24.36M  3.90MB/s    in 13s     

2024-05-27 15:59:24 (1.83 MB/s) - ‘dataset_tsmc2014.zip’ saved [25546284/25546284]

Archive:  dataset_tsmc2014.zip
   creating: dataset_tsmc2014/
  inflating: dataset_tsmc2014/dataset_TSMC2014_NYC.txt  
  inflating: dataset_tsmc2014/dataset_TSMC2014_readme.txt  
  inflating: dataset_tsmc2014/dataset_TSMC2014_TKY.txt  


In [2]:
import pandas as pd
columns = ['User ID',
 'Venue ID',
 'Venue category ID',
 'Venue category name',
 'Latitude',
 'Longitude',
 'Timezone',
 'UTC time']

In [3]:
NUMCHECKIN = 300

df = pd.read_csv('dataset_tsmc2014/dataset_TSMC2014_NYC.txt', sep='\t', encoding='latin-1', names=columns)
df = df.sample(NUMCHECKIN)
print(df)

        User ID                  Venue ID         Venue category ID  \
149759      655  4bd6e7467b1876b0b8308d86  4bf58dd8d48988d126941735   
89993      1060  4f4657cbe4b0f8695e6d26f1  4bf58dd8d48988d124941735   
110274       16  4b4f2c9ff964a52035fc26e3  4bf58dd8d48988d1e0931735   
157447      597  49e4ec34f964a5206e631fe3  4bf58dd8d48988d1fd941735   
47592       880  4b80fc8df964a520489430e3  4bf58dd8d48988d1c5941735   
...         ...                       ...                       ...   
188197      114  3fd66200f964a520dfe51ee3  4bf58dd8d48988d116941735   
105134      354  4bdc9fb73904a59345084f9e  4f2a25ac4b909258e854f55f   
5602        474  4bfdb176b68d0f478e30e957  4bf58dd8d48988d1fc941735   
28311       973  4acbe67af964a52044c820e3  4bf58dd8d48988d146941735   
65276       392  4ab38f3df964a520ca6d20e3  4bf58dd8d48988d129951735   

        Venue category name   Latitude  Longitude  Timezone  \
149759  Government Building  40.714978 -74.005416      -240   
89993                

In [4]:
from sklearn.preprocessing import LabelEncoder


le = LabelEncoder()
df['Venue ID'] = le.fit_transform(df['Venue ID'])
df['Venue category ID'] = le.fit_transform(df['Venue category ID'])
df.drop(['Venue category name'],axis=1,inplace=True)
print(df)

        User ID  Venue ID  Venue category ID   Latitude  Longitude  Timezone  \
149759      655       146                 28  40.714978 -74.005416      -240   
89993      1060       261                 27  40.752896 -73.974144      -240   
110274       16       108                 87  40.863842 -73.975110      -240   
157447      597        33                105  40.726966 -74.038324      -240   
47592       880       122                 74  40.790151 -73.948373      -240   
...         ...       ...                ...        ...        ...       ...   
188197      114         5                 16  40.724013 -73.996263      -300   
105134      354       147                118  40.714249 -73.953760      -240   
5602        474       151                103  40.606173 -73.986294      -240   
28311       973        73                 40  40.722307 -73.987352      -240   
65276       392        70                 29  40.704359 -74.190717      -240   

                              UTC time 

In [5]:
import math

def distance_between_coordinates(lat1, lon1, lat2, lon2):
    radius = 6371  # Radius of the Earth in km

    # Convert latitude and longitude from degrees to radians
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    # Calculate the differences in latitude and longitude<
    d_lat = lat2 - lat1
    d_lon = lon2 - lon1

    # Calculate the distance using the Haversine formula
    a = math.sin(d_lat/2) * math.sin(d_lat/2) + math.cos(lat1) * math.cos(lat2) * math.sin(d_lon/2) * math.sin(d_lon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = radius * c

    return distance

In [6]:
print(distance_between_coordinates(40.719810,-74.002581,40.735981, -74.029309))

2.8819941850086326


In [7]:
#build an alternative temporal graph
#two pois are connected if users go on average to them in the seme timeframe

df['localTime'] = pd.to_datetime(df['UTC time']) + pd.to_timedelta(df['Timezone'], unit = 'm')

df['localDayofWeek'] = df.localTime.dt.dayofweek
df['localHour'] = df.localTime.dt.hour






<ipython-input-7-f85843e15d2c>:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['localTime'] = pd.to_datetime(df['UTC time']) + pd.to_timedelta(df['Timezone'], unit = 'm')


In [8]:
venuesdf = df[['Venue ID','Latitude','Longitude','Venue category ID']].drop_duplicates(subset=['Venue ID'])
numvenues = len(venuesdf)
print(venuesdf)


        Venue ID   Latitude  Longitude  Venue category ID
149759       146  40.714978 -74.005416                 28
89993        261  40.752896 -73.974144                 27
110274       108  40.863842 -73.975110                 87
157447        33  40.726966 -74.038324                105
47592        122  40.790151 -73.948373                 74
...          ...        ...        ...                ...
58965         68  40.956343 -74.069028                105
188197         5  40.724013 -73.996263                 16
5602         151  40.606173 -73.986294                103
28311         73  40.722307 -73.987352                 40
65276         70  40.704359 -74.190717                 29

[289 rows x 4 columns]


In [9]:
#!pip3 install torch --index-url https://download.pytorch.org/whl/${CUDA}
import torch
print(torch.__version__)
print(torch.version.cuda)
!pip install torch-geometric \
  torch-sparse \
  torch-scatter \
  torch-cluster \
  -f https://pytorch-geometric.com/whl/torch-2.2.0+cu121.html

2.3.0+cu121
12.1
Looking in links: https://pytorch-geometric.com/whl/torch-2.2.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 35.5 MB/s eta 0:00:00


In [10]:
import networkx as nx
venuesdict = venuesdf.to_dict('records')

i = 0
nodeIds = {}
node_attr = []

for el in venuesdict:
  #print(el['Venue ID'])
  nodeIds[el['Venue ID']] = i
  node_attr.append([float(el['Venue category ID'])])
  i+=1

print(len(node_attr))


289


In [11]:
#import torch
sequences = {}
count = {}

MINVISIT = 5


for _, row in df.iterrows():

  if row['User ID'] not in sequences:
    sequences[row['User ID']] = [[row['Venue ID'],row['localHour'],row['localDayofWeek']]]


  else:
    sequences[row['User ID']].append([row['Venue ID'],row['localHour'],row['localDayofWeek']])


  if row['Venue ID'] not in count:
    count[row['Venue ID']] = 1
  else:
    count[row['Venue ID']] += 1

print(sequences)
seq = []
next = []
to_remove = []

for el in sequences:
  entry = [el]
  for i in range(len(sequences[el][len(sequences[el])-1])):
    el2 = sequences[el][len(sequences[el])-1][i]
    #print(el2,i)
    if i == 0:
      el2 = nodeIds[el2]
    #print(el2,i)
    entry.append(el2)
  for i in range(len(sequences[el])-1):
    entry.append(nodeIds[sequences[el][i][0]])
  seq.append(torch.tensor(entry))

  target = torch.zeros(numvenues)
  target[sequences[el][len(sequences[el])-1][0]] = 1
  next.append(target)
  #print(count[sequences[el][len(sequences[el])-1][0]])
  if count[sequences[el][len(sequences[el])-1][0]] < MINVISIT:

    to_remove.append(sequences[el][len(sequences[el])-1][0])
    #seq.append(torch.tensor(entry))
    #next.append(target)

seq=torch.nn.utils.rnn.pad_sequence(seq,batch_first=True)
#seq=torch.nn.utils.rnn.pad_sequence(seq)

print(len(to_remove))

#to_remove_new = []

#for el in to_remove:

#  el = nodeIds[el]
#  to_remove_new[]

print(seq)
print(len(next))



{655: [[146, 12, 3]], 1060: [[261, 10, 3]], 16: [[108, 9, 4]], 597: [[33, 11, 5]], 880: [[122, 1, 4]], 999: [[18, 19, 6], [62, 9, 4]], 422: [[233, 13, 1]], 445: [[281, 2, 4], [225, 11, 0]], 837: [[77, 0, 6], [209, 1, 4], [158, 15, 2]], 502: [[83, 13, 2]], 440: [[55, 17, 0]], 715: [[93, 8, 2]], 773: [[282, 4, 0]], 372: [[243, 10, 6], [40, 15, 6], [182, 18, 0], [21, 23, 4]], 1073: [[28, 23, 6]], 666: [[286, 20, 3]], 85: [[52, 11, 5]], 673: [[171, 1, 1]], 780: [[2, 0, 6]], 985: [[131, 11, 6]], 761: [[150, 20, 5]], 527: [[127, 17, 5], [246, 13, 3]], 852: [[31, 15, 6]], 137: [[86, 18, 0]], 964: [[154, 7, 5]], 1018: [[7, 14, 6]], 756: [[102, 10, 4]], 124: [[78, 0, 5], [133, 10, 3]], 84: [[184, 17, 6], [280, 8, 6]], 943: [[274, 19, 5], [25, 21, 2]], 1082: [[232, 6, 0]], 701: [[92, 1, 0]], 70: [[16, 8, 2], [207, 5, 1]], 99: [[145, 13, 5]], 1012: [[222, 6, 5]], 974: [[221, 23, 0], [228, 19, 6]], 69: [[67, 15, 0], [236, 22, 6]], 737: [[253, 23, 3]], 620: [[160, 12, 4]], 1080: [[30, 23, 6]], 64: 

In [12]:
df = df[~df['Venue ID'].isin(to_remove)]
venuesdf = venuesdf[~venuesdf['Venue ID'].isin(to_remove)]
venuesdict = venuesdf.to_dict('records')
print(df)
print(venuesdf)

        User ID  Venue ID  Venue category ID   Latitude  Longitude  Timezone  \
31301       999        18                 56  40.723711 -73.992911      -240   
166296      445       281                 70  40.827397 -73.945516      -240   
141180      837        77                 24  40.633577 -74.006455      -240   
64627       372       243                 50  40.742715 -73.988847      -240   
16553       527       127                104  40.736950 -73.877220      -240   
210921      124        78                 25  40.740636 -74.007780      -300   
5934         84       184                 91  40.639595 -73.979155      -240   
192243      943       274                  3  40.729662 -74.047638      -300   
57106        70        16                100  40.758538 -73.985603      -240   
95589       974       221                 48  40.827375 -73.952930      -240   
54160        69        67                 42  40.709220 -73.869469      -240   
146675     1039       218               

In [13]:
#fast spatial graph construction

import networkx as nx
venuesdict = venuesdf.to_dict('records')
#print(venuesdict)

#for el in venuesdict:
#  print((el,venuesdict[el]))
'''
G = nx.Graph()

distances = []

from tqdm import tqdm

for i in tqdm(range(len(venuesdict) - 1)):
    if not G.has_node(venuesdict[i]['Venue ID']):
        G.add_node(venuesdict[i]['Venue ID'], category=venuesdict[i]['Venue category ID'])

    for j in range(i + 1, len(venuesdict)):
        if not G.has_node(venuesdict[j]['Venue ID']):
            G.add_node(venuesdict[j]['Venue ID'], category=venuesdict[i]['Venue category ID'])

        if distance_between_coordinates(venuesdict[i]['Latitude'], venuesdict[i]['Longitude'],
                                        venuesdict[j]['Latitude'], venuesdict[j]['Longitude']) < 1:
            G.add_edge(venuesdict[i]['Venue ID'], venuesdict[j]['Venue ID'])
'''
from tqdm import tqdm
import torch
from torch_geometric.data import Data

edge_index = []
edges_attr = []

for i in tqdm(range(len(venuesdict) - 1)):

   #if venuesdict[i]['Venue ID'] not in node_attr:
   #  node_attr.append(venuesdict[i]['Venue ID'])

   for j in range(i + 1, len(venuesdict)):
    #  if venuesdict[j]['Venue ID'] not in node_attr:
    #     node_attr.append(venuesdict[j]['Venue ID'])


      if distance_between_coordinates(venuesdict[i]['Latitude'], venuesdict[i]['Longitude'],
                                    venuesdict[j]['Latitude'], venuesdict[j]['Longitude']) < 1:
        edge_index.append([nodeIds[venuesdict[i]['Venue ID']], nodeIds[venuesdict[j]['Venue ID']]])
        edges_attr.append([1])

spatG = Data(x=torch.tensor(node_attr), edge_index=torch.LongTensor(edge_index).t().contiguous(), edge_attr=torch.tensor(edges_attr))
print(spatG)


100%|██████████| 41/41 [00:00<00:00, 10731.81it/s]

Data(x=[289, 1], edge_index=[2, 20], edge_attr=[20, 1])


In [14]:
import pickle

print(spatG)

#spatpG = G

pickle.dump(spatG,open('spatG.pickle','wb'))

#data = pickle.load(open('spatG.pickle','rb'))

Data(x=[289, 1], edge_index=[2, 20], edge_attr=[20, 1])


In [15]:
#nx.draw(G)
print(spatG)

Data(x=[289, 1], edge_index=[2, 20], edge_attr=[20, 1])


In [16]:

#sequantial relation graph
#build a graph that captures the sequence of actions made by the users

import networkx as nx

edge_index = []
edges_attr = []

usrloc = {}

#seqG = nx.Graph()

for _, row in df.iterrows():

  #if not seqG.has_node(row['Venue ID']):
  #  seqG.add_node(row['Venue ID'], category=row['Venue category ID'])

  if row['User ID'] not in usrloc:
    usrloc[row['User ID']] = row['Venue ID']

  else:
    #seqG.add_edge(usrloc[row['User ID']],row['Venue ID'])
    edge_index.append([nodeIds[usrloc[row['User ID']]], nodeIds[row['Venue ID']]])
    edges_attr.append(1)
    usrloc[row['User ID']] = row['Venue ID']


seqG = Data(x=torch.tensor(node_attr), edge_index=torch.tensor(edge_index).t().contiguous(), edge_attr=torch.tensor(edges_attr))
print(seqG)






Data(x=[289, 1], edge_index=[2, 6], edge_attr=[6])


In [17]:
import pickle


print(seqG)

pickle.dump(seqG,open('tempG.pickle','wb'))

#data = pickle.load(open('tempG.pickle','rb'))

Data(x=[289, 1], edge_index=[2, 6], edge_attr=[6])


In [18]:
import math
groupeddf = df.groupby(['Venue ID','Venue category ID'],as_index=False)
avghour = groupeddf['localHour'].mean()
avgday = groupeddf['localDayofWeek'].mean()

print(avghour)

timedf = pd.concat([avghour,avgday['localDayofWeek']],axis=1)



print(timedf)

hours = {}
week = {}

#week['workdays'] = []
#week['weekend'] = []


for _, row in timedf.iterrows():

  hour = math.floor(row['localHour'])
  day = math.floor(row['localDayofWeek'])

  if hour not in hours:
    #hours[hour] = [[row['Venue ID'],row['Venue category ID']]]
    hours[hour] = [[row['Venue ID'],row['Venue category ID']]]

  else:
    hours[hour].append([row['Venue ID'],row['Venue category ID']])

  #if 1 <= day <=4:
  #  week['workdays'].append([row['Venue ID'],row['Venue category ID']])
  #else:
  #  week['weekend'].append([row['Venue ID'],row['Venue category ID']])

  if day not in week:
    #hours[hour] = [[row['Venue ID'],row['Venue category ID']]]
    week[day] = [[row['Venue ID'],row['Venue category ID']]]

  else:
    week[day].append([row['Venue ID'],row['Venue category ID']])






    Venue ID  Venue category ID  localHour
0          4                 85       13.0
1         16                100        8.0
2         18                 56       19.0
3         38                 20       18.0
4         40                 61       15.0
5         54                 59       18.0
6         67                 42       15.0
7         71                 45       12.0
8         72                 92       18.0
9         77                 24        0.0
10        78                 25        0.0
11        91                 50        0.0
12       100                 87       11.0
13       104                104       19.0
14       110                  0       16.0
15       127                104       17.0
16       128                 31        8.0
17       136                104       15.0
18       142                 64       12.0
19       149                  9       16.0
20       153                 41       18.0
21       182                 56       18.0
22       18

In [19]:
#dayG = nx.Graph()
#hourG = nx.Graph()

edge_index = []
edges_attr = []

for el in hours:
   for u in hours[el]:

    #if not hourG.has_node(u[0]):
    #  hourG.add_node(u[0], category=u[1])

    for v in hours[el]:

      if u != v:

        #if not hourG.has_node(u[0]):
        #  hourG.add_node(v[0], category=v[1])

        #hourG.add_edge(u[0],v[0])
        edge_index.append([nodeIds[u[0]], nodeIds[v[0]]])

hourG = Data(x=torch.tensor(node_attr), edge_index=torch.tensor(edge_index).t().contiguous(), edge_attr=torch.tensor(edges_attr))
print(hourG)




Data(x=[289, 1], edge_index=[2, 84], edge_attr=[0])


In [20]:
pickle.dump(hourG,open('hourG.pickle','wb'))

#data = pickle.load(open('tempG.pickle','rb'))

In [21]:
edge_index = []
edges_attr = []


for el in week:
   for u in week[el]:

    #if not dayG.has_node(u[0]):
    #  dayG.add_node(u[0], category=u[1])

    for v in week[el]:

      if u != v:

        #if not dayG.has_node(u[0]):
        #  dayG.add_node(v[0], category=v[1])

        #dayG.add_edge(u[0],v[0])
        edge_index.append([nodeIds[u[0]], nodeIds[v[0]]])

dayG = Data(x=torch.tensor(node_attr), edge_index=torch.tensor(edge_index).t().contiguous(), edge_attr=torch.tensor(edges_attr))
print(dayG)

Data(x=[289, 1], edge_index=[2, 260], edge_attr=[0])


In [22]:
pickle.dump(dayG,open('dayG.pickle','wb'))

In [23]:
!pip install pytorch-lightning
import pytorch_lightning as pl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.3/802.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 40.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-man

In [24]:
from sklearn.model_selection import train_test_split

batchsize = 32


#train_x,test_x,train_y,test_y = train_test_split(seq,torch.stack(next),test_size=0.2,stratify=torch.stack(next))
train_x,test_x,train_y,test_y = train_test_split(seq,torch.stack(next),test_size=0.2)
#train_x,val_x,train_y,val_y = train_test_split(train_x,train_y,test_size=0.2,stratify=train_y)
train_x,val_x,train_y,val_y = train_test_split(train_x,train_y,test_size=0.2)

trainset = torch.utils.data.TensorDataset(train_x,train_y)
trainloader = torch.utils.data.DataLoader(trainset,batch_size=batchsize,shuffle=True)

valset = torch.utils.data.TensorDataset(val_x,val_y)
valloader = torch.utils.data.DataLoader(valset,batch_size=batchsize,shuffle=True)

testset = torch.utils.data.TensorDataset(test_x,test_y)
testloader = torch.utils.data.DataLoader(testset,batch_size=batchsize,shuffle=True)

In [31]:
#import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data

class GCN(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, 64)
        self.conv2 = GCNConv(64, out_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

class Model(nn.Module):
    def __init__(self, input_dim, location_dim, sequence_dim):
        super(Model, self).__init__()

        # User Embedding Layer
        self.user_embedding = nn.Embedding(num_embeddings=input_dim, embedding_dim=64)

        # Location Embedding Layers
        self.gcn1 = GCN(spatG.num_features, 64)
        self.gcn2 = GCN(64, 64)
        self.gcn3 = GCN(64, 64)

        # Sequence Embedding Layer
        self.self_attn = nn.MultiheadAttention(embed_dim=location_dim, num_heads=8)

        # Multihead Attention Layer
        self.multihead_attn = nn.MultiheadAttention(embed_dim=256, num_heads=8)

        # Classifier Layers
        self.linear1 = nn.Linear(256, 128)
        self.linear2 = nn.Linear(128, 64)
        self.linear3 = nn.Linear(64, 1)

    def forward(self, user, lastloc, lasthour, lastday, sequence):
        #print(lastday)
        user_embed = self.user_embedding(user)

        #nodes = spatG.x[lastloc]

        #print(nodes)

        k = 1
        # Location Embeddings
        #nodes,edge_index1,mapping,_ =torch_geometric.utils.k_hop_subgraph(lastloc,k,spatG.edge_index)
        #print(len(nodes))

        #nodes = spatG.x[nodes]

        #print(nodes)

        print(isinstance(spatG.edge_index, torch.Tensor))
        print(spatG.edge_index)



        location_embed1 = self.gcn1(x=spatG.x, edge_index=spatG.edge_index)
        location_embed2 = self.gcn2(location_embed1, edge_index)
        location_embed3 = self.gcn3(location_embed2, edge_index)

        # Sequence Embedding
        sequence_embed, _ = self.self_attn(sequence, sequence, sequence)

        # Concatenate all embeddings
        concat_embed = torch.cat((user_embed, location_embed3, sequence_embed), dim=1)

        # Multihead Attention
        attn_output, _ = self.multihead_attn(concat_embed, concat_embed, concat_embed)

        # Classifier
        out = F.relu(self.linear1(attn_output))
        out = F.relu(self.linear2(out))
        out = self.linear3(out)

        return out

# Define Lightning model using PyTorch Lightning
import pytorch_lightning as pl

class LightningModel(pl.LightningModule):
    def __init__(self, input_dim, location_dim, sequence_dim):
        super(LightningModel, self).__init__()
        self.model = Model(input_dim, location_dim, sequence_dim)

    def forward(self, user, timestamp, location, sequence):
        return self.model(user, timestamp, location, sequence)

    def training_step(self, batch, batch_idx):
        x, y = batch
        #print(x)
        y_pred = self.model(x[:,0],x[:,1],x[:,2],x[:,3],x[:,4:])
        loss = F.binary_cross_entropy(torch.sigmoid(y_pred), y)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        return optimizer

# Example of training the model
input_dim = 10000
location_dim = 64
sequence_dim = 64
model = LightningModel(input_dim, location_dim, sequence_dim)

trainer = pl.Trainer(max_epochs=10)
trainer.fit(model, trainloader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type  | Params
--------------------------------
0 | model | Model | 981 K 
--------------------------------
981 K     Trainable params
0         Non-trainable params
981 K     Total params
3.928     Total estimated model params size (MB)
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (5) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

tensor([[349, 157,  20,   2,   0,   0,   0],
        [371, 225,  13,   6,   0,   0,   0],
        [344, 202,   7,   2,  61, 159,   0],
        [280, 238,  13,   0, 139,   0,   0],
        [293, 183,  12,   2,  47, 146,   0],
        [238, 123,  17,   5,   0,   0,   0],
        [706, 122,   0,   2,   0,   0,   0],
        [ 34,  49,   1,   6,   0,   0,   0],
        [956, 255,  17,   4, 208,   0,   0],
        [753, 100,   7,   1,   0,   0,   0],
        [819,  85,   1,   4,  85,   0,   0],
        [364, 273,  19,   5, 218,   0,   0],
        [427, 181,  19,   4,   0,   0,   0],
        [767, 270,  16,   0,   0,   0,   0],
        [221,  71,  23,   4,   0,   0,   0],
        [ 64,  41,  13,   4,   0,   0,   0],
        [ 69,  93,  22,   6,  37,   0,   0],
        [ 39, 110,  22,   1,   0,   0,   0],
        [715,  11,   8,   2,   0,   0,   0],
        [ 95,  88,   8,   5,  71,   0,   0],
        [965, 140,  15,   3,   0,   0,   0],
        [234, 244,  12,   3,   0,   0,   0],
        [6

ValueError: `MessagePassing.propagate` only supports integer tensors of shape `[2, num_messages]`, `torch_sparse.SparseTensor` or `torch.sparse.Tensor` for argument `edge_index`.